# CSV to ee.FeatureCollection
This script converts the PM-2.5 ground station data (in a csv) to an ee.FeatureCollection for upload to Google Earth Engine.

This is helpful for casting data within table to proper data type. For example, making sure datetime is ee.Date object rather than a string.

In [ ]:
# Accesses Earth Engine API
import ee
ee.Authenticate()
ee.Initialize(project='ee-aspenjkmorgan')

# import csv reader
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Open file using csv library
files = ['/content/drive/MyDrive/INBRE/Rerun_2024/Averaged_PM2.5/PM_ave_1.csv',
         '/content/drive/MyDrive/INBRE/Rerun_2024/Averaged_PM2.5/PM_ave_2.csv']


In [ ]:
# helper function for converting ee.List's of ee elements into a feature collection
def createFC(number):
    point = ee_xy.get(number)
    datetime = ee_datetimes.get(number)
    station = ee_sites.get(number)
    pm25 = ee_rawvalues.get(number)
    return ee.Feature(ee.Geometry.Point(point),
     {'station': station, 'pm25': pm25}).set('system:time_start', datetime)

In [ ]:
for i in range(len(files)):
  with open(files[i], 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # parse the dataframe for date, hour, site name, and raw value
    # convert components to EE objects
    pt_num, xy, sites, rawvalues, datetimes = [], [], [], [], []
    unique_dates = {}

    for row in reader:

      datetimes.append(ee.Date(row['datetime']).millis())
      rawvalues.append(ee.Number(float(row['pm25'])))
      xy.append([ee.Number(float(row['longitude'])), ee.Number(float(row['latitude']))])
      sites.append(ee.String(row['station']))
      pt_num.append(ee.Number(float(row['index'])))

  # put ee objects in ee.Lists
  ee_pt_num = ee.List(pt_num)
  ee_xy = ee.List(xy)
  ee_sites = ee.List(sites)
  ee_rawvalues = ee.List(rawvalues)
  ee_datetimes = ee.List(datetimes)

  # convert to feature collection
  data = ee.FeatureCollection(ee_pt_num.map(createFC))

  # export to Google Earth Engine
  ee.batch.Export.table.toAsset(data, 'PM25', 'projects/ee-aspenjkmorgan/assets/PM_ave_' + str(i + 1)).start()